We are going to use Pearson's R correlation to reccomend an item whic is most similar to the item a user has already chosen - recommend an item which has a review score and THIS correlates to another item a user has already reviewed. 

### Pearson's R Correlation Coeffcicient 
- The Pearson's R correlation coefficient is a measure of a linear correlation between two variables (in this case the 'ratings' of two items).

- As shown below, if you have an R value which is close to 1 or negative 1, then you have a strong linear relationship between two variables/items (in this case an item = film). The closer the R value gets to 0, the furthur apart those two items are in terms of being similar.

- We're going to practise making a recommendation using the Pearson correlation.

### Correlation-Based Collaborative Filtering
Will reccomend a FILM based on how well it correlates with other films, BASED ON/with respect to user ratings.
THis is user-user filtering method - becuase films are reccomended based on similarities based on user reviews. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

<font color=red> Write a little bit about where you got the data from, no discaimer etc. </font>

In [2]:
films = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
# Look at first 5 lines of each dataframe
films.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [81]:
ratings.head()

# Common column of MovieID. RATING is out of 5 - 1-5/5.
# Timestamp column will need formatiign. UserID is duplicate - means a user has rated more than one film.

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Grouping and Ranking Data

In [82]:
# Insight into ratings these films are getting by looking at mean value of all the ratings given to each film.
# Gives us a mean rating for each film.

film_popularity = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
film_popularity.head()

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429


In [83]:
# Evaluate popularity of each film.
# 1. Add a column called 'rating_count' (generates counts for how many reviews each film got).



film_popularity['rating_count'] = pd.DataFrame(ratings.groupby('movieId')['rating'].count())
film_popularity.head()

# Now we have each of the films - their average rating and then the no. of ratings each film got .

,rating,rating_count
movieId,,
1,3.920930,215
2,3.431818,110
3,3.259615,52
4,2.357143,7
5,3.071429,49


In [84]:
# Let's look at a statistical description of this rating dataframe.
film_popularity.describe()

# Shows there are 9724 unique films that have been reviewed.
# Looked at max value for rating count = 329. The most popular film in the dataset has got 329 reviews.

,rating,rating_count
count,9724.000000,9724.000000
mean,3.262448,10.369807
std,0.869874,22.401005
min,0.500000,1.000000
25%,2.800000,1.000000
50%,3.416667,3.000000
75%,3.911765,9.000000
max,5.000000,329.000000


In [85]:
# Lets see which film was the most popular, by sorting data in ascening order.
film_popularity.sort_values('rating_count', ascending=False).head()

# We see our most popular film has a 'movieId: 356'

,rating,rating_count
movieId,,
356,4.164134,329
318,4.429022,317
296,4.197068,307
593,4.161290,279
2571,4.192446,278


In [86]:
# Want to find the name of this film.
# Create a filter which creates a true value for where the movieId=356.
# 1. Create Filter: where the expression is true, we want to get that record from the 'films df'

films[films['movieId']==356]

# Forest Gump is the most highly rated film in our data.

,movieId,title,genres
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War


In [87]:
test = film_popularity.sort_values('rating_count', ascending=False)
test1 = pd.DataFrame(test.loc[:, 'rating_count'])


In [88]:
# test1.head(20)

### Preparing Data for Analysis

- We're going to create a user by item utility matrix. 
- 1. call the pivot table functionb - this function will cross tabulate each user, against each place and ouput a matrix.

In [89]:
films_crosstab = pd.pivot_table(data=ratings, values='rating', index='userId', columns='movieId')
films_crosstab.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It is full of Null Values - this is because people don't review 100's of films, people will only review a few films
Makes the matrix spare.
The numbers are the ratings that each user gave to their respective place that they did review - in cases where they DID give a film review.

How do we use it to find places which are correlated? We can.
First, let's isolate the user ratings from out most popualar film 'Forest Gump'

In [91]:
# Filter Forest Gump ratings so we can see only NON null values.
# remember: FOREST GUMP is the most popular place with 329 ratings lets look at what those ratings are.

ForestGump_ratings = films_crosstab[356]
ForestGump_ratings[ForestGump_ratings>=0]

# have 329 review scored between 0-5

userId
1      4.0
6      5.0
7      5.0
8      3.0
10     3.5
11     5.0
14     4.0
15     5.0
16     3.5
17     5.0
18     4.5
19     2.0
21     4.5
22     5.0
24     4.5
26     3.0
27     5.0
28     4.0
29     4.5
33     5.0
34     4.0
37     4.0
38     3.0
41     2.0
42     5.0
43     5.0
45     5.0
47     4.5
49     4.0
50     3.0
      ... 
567    3.0
568    5.0
569    3.0
570    4.0
572    4.0
573    4.5
577    5.0
579    5.0
580    4.0
581    4.5
583    4.0
584    5.0
587    4.0
588    3.0
589    5.0
590    5.0
591    4.0
592    5.0
593    4.0
596    3.5
597    5.0
599    3.5
600    4.0
602    3.0
603    3.0
605    3.0
606    4.0
608    3.0
609    4.0
610    3.0
Name: 356, Length: 329, dtype: float64

### Evaluating Similarity Based on Correlation

To find correlation between Forest Gump (our most popular film) and each of the other thousands of films.
- 1. call the core_with() method of of our films_crosstab
- 2. And then PASS it the Forest_Gump rating series. 

- 3. This will generate a Pearson R correlation coefficient between Forest_Gump (most popular movie) and each other film which has been reviewed in the dataset.
- 4. BEAR IN MIND this correlation is based on similarities and user reviews that were given to each place.

In [92]:
# Name it 'similar_to_ForestGump' becuase looking for films similar to Forest Gump.
similar_to_ForestGump = films_crosstab.corrwith(ForestGump_ratings)

/Users/jmi61/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2392: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/jmi61/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [93]:
# This will be returned as a matrix - and want a dataframe.
similar_to_ForestGump = pd.DataFrame(similar_to_ForestGump, columns=['PearsonR'])

# Drop Null values
similar_to_ForestGump.dropna(inplace=True)

similar_to_ForestGump.head()

,PearsonR
movieId,
1,0.303465
2,0.367247
3,0.534682
4,0.388514
5,0.349541


Here we have a dataframe which contains each movieID and a Pearson's R Correlation Coefficient, which indicates how well each film correlates with Forest Gump based on user ratings.
BUT - lets say we have some places which are really well correlated (high similarity) with Forest Gump, but actually only 2 or 3 people had submitted a rating (low number of ratings).

Then these films might not actually be all that similar to Forest Gump. They would have similar RATINGS as Forest Gump but would not be popular films. In this case the correlation wouldn't be significant. 

THEREFORE we need to take into account how popular each film is IN ADDITION TO - how well the review scores correlate with the ratings that were given to other places in the dataset.
Lets do this.

In [94]:
# join ratings dataframe with similar_to_ForestGump dataframe.

similarity_summary = similar_to_ForestGump.join(film_popularity['rating_count'])

In [95]:
# Create filter so you can abstract only places from the data frame which have at least 50 reviews 
# And for THESE films lets look at the pearson's R correlation sorted is descending order.
# Films with over 50 ratings - sort in descending order according to Pearson's R.
# View first 10 rows of data.

similarity_summary[similarity_summary['rating_count']>=40].sort_values('PearsonR', ascending=False).head(10)

,PearsonR,rating_count
movieId,,
356,1.000000,329
62,0.652144,80
1032,0.577629,40
2003,0.563334,41
3448,0.562116,43
48,0.550118,68
3,0.534682,52
899,0.527961,47
252,0.527046,43


In [47]:
# We've sorted the dataframe in descending order by correlation.
# We now have a list of TOP reviewed places similar to Forest Gump.

In [ ]:
# Lets find out what the names of the top 5 similar films to Forest Gump are
